In [0]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import mlflow
import mlflow.sklearn

In [0]:

df = spark.table("workspace.ecommerce.ecommerce_events_delta")

purchase_df = (
    df
    .select("price", "event_type")
    .withColumn(
        "label", (df.event_type == "purchase").cast("int")
    ).dropna().toPandas()
)
   

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-6426400283418337>, line 7
      1 df = spark.table("workspace.ecommerce.ecommerce_events_delta")
      3 purchase_df = (
      4     df
      5     .select("price", "event_type")
      6     .withColumn(
----> 7         "label", (df.event_type == "purchase").cast("int")
      8     ).dropna().toPandas()
      9 )

File /databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/dataframe.py:1736, in DataFrame.__getattr__(self, name)
   1728     raise AttributeError(
   1729         f"'{self.__class__.__name__}' object has no attribute '{name}'. If you are "
   1730         f"attempting to reference a column named '{name}' then please use the df['{name}'] "
   1731         f"syntax instead: the df.{name} syntax is unsupported for fields with names that "
   1732         "match IPython's rich display method names."
 

In [0]:

X = purchase_df[["price"]]
y = purchase_df["label"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
     

com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$5(SequenceExecutionState.scala:139)
	at scala.Option.getOrElse(Option.scala:201)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:139)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:136)
	at scala.collection.immutable.Range.foreach(Range.scala:192)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:136)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:721)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:441)
	at scala.Option.getOrElse(Option.scala:201)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:441)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.can

In [0]:
with mlflow.start_run(run_name="logistic_reg_v1"):

    # Parameters
    mlflow.log_param("model_type", "LogisticRegression")
    mlflow.log_param("test_size", 0.2)
    mlflow.log_param("features", "price")

    # Train
    model = LogisticRegression()
    model.fit(X_train, y_train)

    # Predict & evaluate
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)

    # Metric
    mlflow.log_metric("accuracy", accuracy)

    # Log model
    mlflow.sklearn.log_model(model, "model")

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-6426400283418339>, line 10
      8 # Train
      9 model = LogisticRegression()
---> 10 model.fit(X_train, y_train)
     12 # Predict & evaluate
     13 y_pred = model.predict(X_test)

NameError: name 'X_train' is not defined

In [0]:
print(f"Model Accuracy: {accuracy:.4f}")

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-6426400283418340>, line 1
----> 1 print(f"Model Accuracy: {accuracy:.4f}")

NameError: name 'accuracy' is not defined

In [0]:
from mlflow.models import infer_signature

# Infer signature from training data
signature = infer_signature(X_train, model.predict(X_train))

mlflow.sklearn.log_model(
    model,
    artifact_path="model",
    signature=signature,
    registered_model_name="ecommerce_purchase_logreg"
)
     